In [1]:
import pandas as pd
import numpy as np
import itertools as it
import os,re
from collections import defaultdict

In [114]:
import pandas as pd
import numpy as np
import itertools as it
import os,re
from collections import defaultdict

import sys
sys.path.append("/cluster/bh0085/mybio")
import ipnb_utils

#dependencies from elsewhere in the C libraries
from c_create_dictionaries import list_umis
from _config import DATA_DIR, OUT_PLACE

inp_dir = os.path.join(OUT_PLACE, "b0_demultiplex_transcripts")


In [115]:
split = "1"
exp = exps[0]
f_infos = dict([[fn,re.compile("(?P<exp>[^/]*)/R(?P<rnum>[12])_(?P<split>\d+).fa").search(fn).groupdict()] 
                 for fn in fa_found])

print split,  exp
f1 = [k for k,v in f_infos.items() if v["rnum"] == "1" and v["split"]==split and v["exp"]==exp][0] 
f2 = [k for k,v in f_infos.items() if v["rnum"] == "2" and v["split"]==split and v["exp"]==exp][0] 


nl = 0
nacc = 0
nnreject = 0
umis = []
t_umis = []
with open(f1) as f1open:
  with open(f2) as f2open:
    i = -1
    while 1:
        i+=1
        try:
            l1 = f1open.next()
            l2 = f2open.next()
        except StopIteration:
            break

        if i %2 == 0: continue
        if "N" in l2[:16]:
            nnreject +=1
            continue
        umi = l2[:15].strip()
        umis.append(umi)          
        t_umi = l1[:10].strip()
        t_umis.append(t_umi)
        nacc+=1


1 HCT116_WT_BR1


In [117]:
#THIS FUNCTION GETS TRANSCRIPTS!!!!


fa_found = []
def wfun(arg,dirname,fnames):
    for f in fnames:
        if f[-2:]=="fa":
            fa_found.append(os.path.join(dirname,f))
os.path.walk("../out/b0_demultiplex_transcripts/",wfun,None)
  

def list_umis(pool_prefix="Undetermined_AH3W5GBGX9_S0_L001", split="0",exp="HCT116_Gemcitabine2uM_BR2"):
    f_infos = dict([[fn,re.compile("(?P<pool_prefix>[^/]*)/(?P<exp>[^/]*)/R(?P<rnum>[12])_(?P<split>\d+).fa").search(fn).groupdict()] 
                     for fn in fa_found])
    #print(fa_found)
    #print(f_infos)
    f1 = [k for k,v in f_infos.items() if v["rnum"] == "1" and v["split"]==split and v["exp"]==exp and v["pool_prefix"]==pool_prefix][0] 
    f2 = [k for k,v in f_infos.items() if v["rnum"] == "2" and v["split"]==split and v["exp"]==exp and v["pool_prefix"]==pool_prefix][0] 

    skipcount = 0
    nl = 0
    nacc = 0
    nnreject = 0
    umis = []
    t_umis = []

    print(f1, f2)
    with open(f1) as f1open:
      with open(f2) as f2open:
        i = -1
        while 1:
            i+=1
            try:
                l1 = f1open.next()
                if len(l1)< 5: 
                    
                    print(f1)
                    print(i)
                    print(l1)
                    print("skipping")
                    skipcount+=1
                    l1 = f1open.next()

                l2 = f2open.next()
                if len(l2)< 5: 
                    
                
                    print(f2)
                    print(i)
                    print(l2)
                    print("skipping2")
                    l2 = f2open.next()                

            except StopIteration:
                break

            if i %2 == 0: continue
            if "N" in l2[:16]:
                nnreject +=1
                continue


            umi = l2[:15].strip()
            umis.append(umi)          
            t_umi = l1[:10].strip()
            t_umis.append(t_umi)

            if( ">" in t_umi) or (">" in umi):
                raise Exception()
            nacc+=1
    print("SKIPPED {}".format(skipcount))
    return umis, t_umis, {"nacc":nacc, "nnreject":nnreject}

In [112]:
os.path.join(inp_dir, pool_prefix)

'/cluster/bh0085/prj/starr_all/starr/out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001'

In [118]:
all_bcs2umis = dict()


pool_prefixes = os.listdir(inp_dir)
for pool_prefix in pool_prefixes:
    exps = os.listdir(os.path.join(inp_dir,pool_prefix))
    total_count = 0 

    
    for exp in exps:
        print (exp,pool_prefix)
        
        all_bcs2umis[exp] = defaultdict(list)
        
        for split in [str(e) for e in range(30)]:
            print split
            bcs,umis,stats = list_umis(pool_prefix = pool_prefix,split = split, exp =exp)
            for i,bc in enumerate(bcs):
                all_bcs2umis[exp][bc].append(umis[i])


('HCT116_WT_BR1', 'Undetermined_AH3W5GBGX9_S0_L001')
0
('../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R1_0.fa', '../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R2_0.fa')
SKIPPED 0
1
('../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R1_1.fa', '../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R2_1.fa')
SKIPPED 0
2
('../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R1_2.fa', '../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R2_2.fa')
SKIPPED 0
3
('../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R1_3.fa', '../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R2_3.fa')
SKIPPED 0
4
('../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116_WT_BR1/R1_4.fa', '../out/b0_demultiplex_transcripts/Undetermined_AH3W5GBGX9_S0_L001/HCT116

KeyboardInterrupt: 

In [31]:
import json

In [33]:
for k,v in all_bcs2umis.iteritems():
    print k
    with open("../data/all_{}_bcs2umis.json".format(k), "w") as f:
        f.write(json.dumps(v))
    

DLD1_WT_BR2
DLD1_WT_BR1
HCT116_WT_BR2
HCT116_WT_BR1
U2OS_WT_BR1
other
U2OS_WT_BR2
HCT116_Gemcitabine2uM_BR2
HCT116_Gemcitabine2uM_BR1


In [40]:
import json
with open("../data/all_bcs2umis.json", "w") as f:
    f.write(json.dumps(all_bcs2umis))

In [ ]:
all_bcs2umis

In [36]:
print "HI"

HI


# PLOTTING